In [1]:
# extract_activations.py
from activations_store import ActivationsStore  # Adjust import path as necessary
from transformer_lens import HookedTransformer
from datasets import load_dataset
import torch
import math


/opt/conda/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for c4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/c4
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


/root/.cache/huggingface/modules/datasets_modules/datasets/c4/584d57ebe81c209b6c7f31727066d2c4b4bba37cb7092cdd83083d5ec11207db/c4.py:53: FutureWarning: Dataset 'c4' is deprecated and will be deleted. Use 'allenai/c4' instead.
  warnings.warn(


./config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

./model_final.pth:   0%|          | 0.00/227M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.04M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

Loaded pretrained model NeelNanda/SoLU_2L512W_C4_Code into HookedTransformer
torch.Size([16, 256, 2, 512])
shape is: (batches, context, num_layers, d_in)
torch.Size([16, 256, 2, 512])
shape is: (batches, context, num_layers, d_in)
torch.Size([16, 256, 2, 512])
shape is: (batches, context, num_layers, d_in)
torch.Size([16, 256, 2, 512])
shape is: (batches, context, num_layers, d_in)
torch.Size([16, 256, 2, 512])
shape is: (batches, context, num_layers, d_in)
torch.Size([16, 256, 2, 512])
shape is: (batches, context, num_layers, d_in)
torch.Size([16, 256, 2, 512])
shape is: (batches, context, num_layers, d_in)
torch.Size([16, 256, 2, 512])
shape is: (batches, context, num_layers, d_in)
torch.Size([16, 256, 2, 512])
shape is: (batches, context, num_layers, d_in)
torch.Size([16, 256, 2, 512])
shape is: (batches, context, num_layers, d_in)
torch.Size([16, 256, 2, 512])
shape is: (batches, context, num_layers, d_in)
torch.Size([16, 256, 2, 512])
shape is: (batches, context, num_layers, d_in)

In [ ]:
#Let's load an existing sae to initialise the weights

import torch 
from sae_lens import LMSparseAutoencoderSessionloader
from huggingface_hub import hf_hub_download

layer = 0 # pick a layer you want.
REPO_ID = "jbloom/GPT2-Small-SAEs"
FILENAME = f"final_sparse_autoencoder_gpt2-small_blocks.{layer}.hook_resid_pre_24576.pt"
path = hf_hub_download(repo_id=REPO_ID, filename=FILENAME)
model, sparse_autoencoder, activation_store = LMSparseAutoencoderSessionloader.load_session_from_pretrained(
    path = path
)
sparse_autoencoder.eval()

pretrained_weights = {}
for i, sae in enumerate(sparse_autoencoder):
    sae_weights = {}  # Dictionary for current SAE weights
    print(f"Parameters for SAE {i}:")
    for name, param in sae.named_parameters():
        sae_weights[name] = param.data.clone()  # Clone the parameter tensor
        print(f"Layer Name: {name}")
        print(f"Values: \n{param.data}")

    # Store weights for each SAE indexed by its position
    pretrained_weights[f"SAE_{i}"] = sae_weights


In [10]:
from config import LanguageModelSAERunnerConfig
from dataclasses import dataclass
from typing import Any, cast
from activations_store import ActivationsStore
from transformer_lens import HookedTransformer
import einops
import torch
from sae_group import SAEGroup

import wandb
from sparse_autoencoder import SparseAutoencoder
from train_sae_on_language_model import train_sae_on_language_model


config = LanguageModelSAERunnerConfig(
    model_name='gpt2-small',
    hook_point="blocks.{layer}.hook_resid_post", 
    hook_point_layer= [0,1], 
    hook_point_head_index=None,
    dataset_path="NeelNanda/openwebtext-tokenized-9b",
    is_dataset_tokenized=True,
    context_size=128,
    use_cached_activations=False,
    cached_activations_path = (
        None  # Defaults to "activations/{dataset}/{model}/{full_hook_name}_{hook_point_head_index}"
    ),

    lr_scheduler_name = (
        "constant"  # constant, cosineannealing, cosineannealingwarmrestarts
    ),
    d_in=768,
    n_batches_in_buffer=2,
    total_training_tokens=2_000_000,
    store_batch_size=32,
    train_batch_size=2000, 
    device = "cuda" if torch.cuda.is_available() else "cpu", 
    seed=42,
    dtype=torch.float32,
    prepend_bos=True,
    b_dec_init_method='geometric_median',
    expansion_factor=32,
    from_pretrained_path=None,
    d_sae=24576,
    l1_coefficient=8e-05,
    lp_norm=1, lr=0.0004,
    lr_warm_up_steps=5000,
    lr_end=None,
    lr_decay_steps=0,
    n_restart_cycles=1,
    use_ghost_grads=True,
    feature_sampling_window=1000, 
    dead_feature_window=5000,
    dead_feature_threshold=1e-08,
    log_to_wandb=True,
    wandb_project='Modified_autoencoder_gpt2_tokens_2m',
    run_name='24576-L1-8e-05-LR-0.0004-Tokens-3.000e+08', 
    wandb_entity=None, wandb_log_frequency=100, n_checkpoints=100, checkpoint_path='checkpoints/ut7lhl4q', verbose=True)



model = HookedTransformer.from_pretrained("gpt2")
sparse_autoencoder = SparseAutoencoder(config) 
activation_store =ActivationsStore(config,model)

if config.log_to_wandb:
    wandb.init(project=config.wandb_project, config=cast(Any, config))
    
pretrained_weight =  pretrained_weights[f"SAE_{0}"]
sae_group = SAEGroup(config, pretrained_weight)

sparse_autoencoder = train_sae_on_language_model(
    model,
    sae_group = sae_group,
    activation_store= activation_store,
    batch_size=config.train_batch_size,
    feature_sampling_window=config.feature_sampling_window,
    dead_feature_threshold=config.dead_feature_threshold,
    use_wandb=config.log_to_wandb,
    wandb_log_frequency=config.wandb_log_frequency,
)

if config.log_to_wandb:
    wandb.finish()


Run name: 24576-L1-8e-05-LR-0.0004-Tokens-2.000e+06
n_tokens_per_buffer (millions): 0.008192
Lower bound: n_contexts_per_buffer (millions): 6.4e-05
Total training steps: 1000
Total wandb updates: 10
n_tokens_per_feature_sampling_window (millions): 256.0
n_tokens_per_dead_feature_window (millions): 1280.0
We will reset the sparsity calculation 1 times.
Number tokens in sparsity calculation window: 2.00e+06
Using Ghost Grads.
Loaded pretrained model gpt2 into HookedTransformer


Resolving data files:   0%|          | 0/37 [00:00<?, ?it/s]

Run name: 24576-L1-8e-05-LR-0.0004-Tokens-2.000e+06
n_tokens_per_buffer (millions): 0.008192
Lower bound: n_contexts_per_buffer (millions): 6.4e-05
Total training steps: 1000
Total wandb updates: 10
n_tokens_per_feature_sampling_window (millions): 256.0
n_tokens_per_dead_feature_window (millions): 1280.0
We will reset the sparsity calculation 1 times.
Number tokens in sparsity calculation window: 2.00e+06
Using Ghost Grads.
Run name: 24576-L1-8e-05-LR-0.0004-Tokens-2.000e+06
n_tokens_per_buffer (millions): 0.008192
Lower bound: n_contexts_per_buffer (millions): 6.4e-05
Total training steps: 1000
Total wandb updates: 10
n_tokens_per_feature_sampling_window (millions): 256.0
n_tokens_per_dead_feature_window (millions): 1280.0
We will reset the sparsity calculation 1 times.
Number tokens in sparsity calculation window: 2.00e+06
Using Ghost Grads.
Loaded pretrained weights for W_enc
Loaded pretrained weights for b_enc
Loaded pretrained weights for W_dec
Loaded pretrained weights for b_dec


Training SAE:   0%|          | 0/2000000 [00:00<?, ?it/s]

layer_acts shape torch.Size([2000, 2, 768])
Shape of X should be .... din
torch.Size([2000, 768])
Shape of Y should be .... din
torch.Size([2000, 768])
Mid layer output
tensor([[ 0.0012, -0.0034,  0.0037,  ...,  0.0004,  0.0003,  0.0000],
        [ 0.0009, -0.0021,  0.0000,  ...,  0.0032,  0.0000,  0.0002],
        [ 0.0000,  0.0000,  0.0048,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0012,  0.0000,  0.0000]],
       device='cuda:0', grad_fn=<AddBackward0>)
Shape of X should be .... din
torch.Size([2000, 768])
Shape of Y should be .... din
torch.Size([2000, 768])
Mid layer output
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0.

...

Shape of X should be .... din
torch.Size([32, 128, 768])
Shape of Y should be .... din
torch.Size([32, 128, 768])
Mid layer output
tensor([[[ 5.8580e-03, -4.3162e-02, -1.4040e-01,  ..., -4.8202e-03,
           5.1358e-03, -4.8809e-03],
         [-2.8392e-03,  3.0940e-03,  8.3505e-05,  ..., -4.8202e-03,
          -3.6266e-03, -4.8809e-03],
         [-2.8392e-03,  3.0940e-03,  8.3505e-05,  ..., -4.8202e-03,
          -3.6131e-03, -4.8809e-03],
         ...,
         [-2.8392e-03,  3.0940e-03,  8.3505e-05,  ..., -4.8202e-03,
          -3.6266e-03, -4.8809e-03],
         [-2.8392e-03,  3.0940e-03,  8.3505e-05,  ..., -4.8202e-03,
          -3.6266e-03, -4.8809e-03],
         [-2.8392e-03,  2.3563e-03, -6.8020e-03,  ..., -5.9815e-03,
          -3.1898e-03, -4.8809e-03]],

        [[ 5.8580e-03, -4.3162e-02, -1.4040e-01,  ..., -4.8202e-03,
           5.1358e-03, -4.8809e-03],
         [-2.8392e-03,  3.0940e-03,  8.3505e-05,  ..., -4.8202e-03,
          -3.6266e-03, -4.8809e-03],
         [-2.


100%|██████████| 10/10 [00:03<00:00,  2.63it/s]
1000| MSE Loss 0.030 | L1 0.107: 100%|██████████| 2000000/2000000 [04:54<00:00, 728.29it/s] 

Shape of X should be .... din
torch.Size([32, 128, 768])
Shape of Y should be .... din
torch.Size([32, 128, 768])
Mid layer output
tensor([[[ 5.8580e-03, -4.3162e-02, -1.4040e-01,  ..., -4.8202e-03,
           5.1358e-03, -4.8809e-03],
         [-2.8392e-03,  3.0940e-03,  8.3505e-05,  ..., -4.8202e-03,
          -3.6266e-03, -4.8809e-03],
         [-2.8392e-03,  3.0940e-03,  8.3505e-05,  ..., -4.8202e-03,
          -3.6266e-03, -4.8809e-03],
         ...,
         [-2.8392e-03,  3.0940e-03,  8.3505e-05,  ..., -4.8202e-03,
          -3.6266e-03, -4.8809e-03],
         [-2.8392e-03,  3.0940e-03,  8.3505e-05,  ..., -4.8202e-03,
          -3.6266e-03, -4.8809e-03],
         [-2.8392e-03,  3.0940e-03,  8.3505e-05,  ..., -4.8202e-03,
          -3.6266e-03, -4.8809e-03]],

        [[ 5.8580e-03, -4.3162e-02, -1.4040e-01,  ..., -4.8202e-03,
           5.1358e-03, -4.8809e-03],
         [-2.8392e-03,  3.0940e-03,  8.3505e-05,  ..., -4.8202e-03,
          -3.6266e-03, -4.8809e-03],
         [-2.

1000| MSE Loss 0.030 | L1 0.107: 100%|██████████| 2000000/2000000 [05:13<00:00, 728.29it/s]

details/current_learning_rate_layer0,▁▂▃▃▄▅▆▆▇█
details/current_learning_rate_layer1,▁▂▃▃▄▅▆▆▇█
details/n_training_tokens,▁▂▃▃▄▅▆▆▇█
losses/ghost_grad_loss_layer0,▁▁▁▁▁▁▁▁▁▁
losses/ghost_grad_loss_layer1,▁▁▁▁▁▁▁▁▁▁
losses/l1_loss_layer0,▇▇█▅▆▄▃▃▁▃
losses/l1_loss_layer1,█▇▇▄▄▃▂▂▁▁
losses/mse_loss_layer0,█▄▃▂▂▁▁▁▁▁
losses/mse_loss_layer1,█▆▅▂▂▁▁▁▁▁
losses/overall_loss_layer0,█▄▃▂▂▂▁▁▁▁
losses/overall_loss_layer1,█▆▅▃▃▂▂▂▁▂


In [11]:
import os

# Assuming `sae_group` is your SAEGroup instance and you have the path defined
dir_name = "trained_autoencoders_final"
file_name = "sae_group_gpt_openweb_bloom_init_features.pt"  # or "sae_group.pkl" if you prefer pickle format

# Create the directory if it does not exist
if not os.path.exists(dir_name):
    os.makedirs(dir_name)
    print(f"Directory '{dir_name}' was created.")

# Define the full path for the new file
file_path = os.path.join(dir_name, file_name)

# Use the `save_model` method from SAEGroup class to save the model
sparse_autoencoder.save_model(file_path)

print(f"SAEGroup was saved to '{file_path}'.")


Saved model to trained_autoencoders_final/sae_group_gpt_openweb_bloom_init_features.pt
SAEGroup was saved to 'trained_autoencoders_final/sae_group_gpt_openweb_bloom_init_features.pt'.


In [12]:
print(sparse_autoencoder)

In [2]:
from config import LanguageModelSAERunnerConfig
from dataclasses import dataclass
from typing import Any, cast
from activations_store import ActivationsStore
from transformer_lens import HookedTransformer
import einops
import torch
from sae_group import SAEGroup
pretrained_path = 'trained_autoencoders_final/sae_group_gpt_openweb_bloom_init_features.pt'

# Load the pretrained SAEGroup
pretrained_sae_group = SAEGroup.load_from_pretrained(pretrained_path)


autoencoder_model = pretrained_sae_group.autoencoders[0]

In [3]:
for name, param in autoencoder_model.named_parameters():
    print(f"Layer Name: {name}")
    print(f"Parameter Shape: {param.size()}")
    print(f"Values: \n{param}\n")

Layer Name: W_enc
Parameter Shape: torch.Size([768, 24576])
Values: 
Parameter containing:
tensor([[-0.0051,  0.0068, -0.0824,  ..., -0.0199, -0.0023, -0.0029],
        [ 0.0234, -0.0057,  0.0462,  ...,  0.0245,  0.0148, -0.0056],
        [-0.0140,  0.0227,  0.0179,  ..., -0.0013,  0.0047,  0.0093],
        ...,
        [ 0.0208,  0.0019,  0.0077,  ...,  0.0342,  0.0064,  0.0240],
        [ 0.0413,  0.0140, -0.0145,  ...,  0.0505,  0.0112,  0.0471],
        [-0.0365, -0.0143,  0.0453,  ..., -0.0390, -0.0205, -0.0078]],
       device='cuda:0', requires_grad=True)

Layer Name: b_enc
Parameter Shape: torch.Size([24576])
Values: 
Parameter containing:
tensor([-0.2882, -0.0174, -0.3979,  ..., -0.0287, -0.0193, -0.0469],
       device='cuda:0', requires_grad=True)

Layer Name: W_mid
Parameter Shape: torch.Size([24576, 24576])
Values: 
Parameter containing:
tensor([[ 2.5715e-02, -5.2542e-05,  9.5891e-03,  ..., -1.3988e-02,
         -1.4744e-02, -1.4625e-02],
        [-1.3369e-02,  1.0868e-02,

In [5]:
layer_transformation = autoencoder_model.W_mid.detach().to('cpu')
decoder = autoencoder_model.W_dec.detach().to('cpu')